In [1]:
import ee

In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [78]:
import geemap as emap

Map = emap.Map(center=[4,-76.5], zoom=13)

In [79]:
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

START_DATE = ee.Date('2020-01-01')
END_DATE = ee.Date('2020-03-01')
MAX_CLOUD_PROBABILITY = 65
region = ee.Geometry.Rectangle(**{'coords': [-76.5, 2.0, -74, 4.0]})

In [80]:
def maskClouds(img):
      clouds = ee.Image(img.get('cloud_mask')).select('probability')
      isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
      return img.updateMask(isNotCloud)

# The masks for the 10m bands sometimes do not exclude bad data at
# scene edges, so we apply masks from the 20m and 60m bands as well.
/# Example asset that needs this operation:
# COPERNICUS/S2_CLOUD_PROBABILITY/20190301T000239_20190301T000238_T55GDP
def maskEdges(s2_img):
      return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

In [81]:
# Filter input collections by desired data range and region.
s2Sr = s2Sr.filterBounds(region).filterDate(START_DATE, END_DATE).map(maskEdges)
s2Clouds = s2Clouds.filterBounds(region).filterDate(START_DATE, END_DATE)

In [82]:
# Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(**{
  'primary': s2Sr,
  'secondary': s2Clouds,
  'condition':
      ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})
})

s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).median()
rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(s2CloudMasked, rgbVis, 'S2 SR masked at ' + str(MAX_CLOUD_PROBABILITY) + '%', True)

In [83]:
Map

Map(center=[4, -76.5], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…